In [1]:
!../data_processing/setup_spacy.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.5 MB/s eta 0:00:000:00:010:00:0101
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import sys; sys.path.append('../data_processing/')
import pandas as pd
from parse_dep import *
import re
from ast import literal_eval
import tokenizations
from datasets import load_dataset

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
bpe_pipeline = get_spacy_pipeline()

In [23]:
default_pipeline = spacy.load("en_core_web_sm")

In [5]:
df = pd.read_csv('wiki_text_test_parsed2.pred.mwe', sep='\t', names=[0,'sentence','d'],)

In [6]:
wiki_text = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")['text']
wiki_text = [w for w in wiki_text if w]

Found cached dataset wikitext (/Users/andrejerkelens/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [7]:
df['d'] = df.apply(lambda x: literal_eval(x['d']), axis=1)

In [8]:
df = pd.concat([df, pd.json_normalize(df.d)],axis=1)
df = df.drop(columns=[0])

In [9]:
df['tokens'] = df['toks'].apply(lambda x: [y[0] for y in x])
df['sentence'] = df['sentence'].apply(lambda x: literal_eval(x).decode('utf-8'))

In [28]:
df["syntactic_distance_tok"] = df["sent"].apply(lambda x: get_syntactic_distance(default_pipeline(x), index=False))
df["syntactic_distance_idx"] = df["sent"].apply(lambda x: get_syntactic_distance(default_pipeline(x), index=True))

---

### Note:

Only piece is to remove underscore from sentences 

In [12]:
def parse_sentences_mwe(dataset, pipeline = "spacy"):
    lines = []
    for d in dataset:
        if pipeline == "spacy":
            doc = nlp(d)
        elif pipeline == "bpe":
            doc = bpe_pipeline(d)
        if doc:
            for sent in doc.sents:
                lines.append(f"{word}\t{word.pos_}\n")
                lines.append("\n")
    return lines

def strip_g(l: list):
    return [i.replace("Ġ", "") for i in l]

In [13]:
_, two2one = tokenizations.get_alignments(df.iloc[1].tokens, list(map(str, list(bpe_pipeline(wiki_text[1]).sents)[0])))

In [14]:
df['sent'] = df['sentence'].apply(lambda x: x.replace("_", " ").replace("~", " "))

In [33]:
df['bpe_tokens'] = df.apply(lambda x: list(map(str, list(bpe_pipeline(x["sent"])))), axis=1)

In [31]:
df.head()

,sentence,d,toks,tags,_,~,lemmas,tokens,sent,syntactic_distance_tok,syntactic_distance_idx
0,= Robert_Boulter =,"{'toks': [['=', 'VERB'], ['Robert', 'PROPN'], ...","[[=, VERB], [Robert, PROPN], [Boulter, PROPN],...","[O, B-GROUP, Ī, O]","[[2, 3]]",[],"[=, robert, boulter, =]","[=, Robert, Boulter, =]",= Robert Boulter =,"{(=, Robert): 2, (=, Boulter): 1, (=, =): 1, (...","{(0, 1): 2, (0, 2): 1, (0, 3): 1, (1, 2): 1, (..."
1,"Robert_Boulter is an English film , television...","{'toks': [['Robert', 'PROPN'], ['Boulter', 'PR...","[[Robert, PROPN], [Boulter, PROPN], [is, AUX],...","[B-PERSON, Ī, O, O, O, O, O, O, O, O, O, O]","[[1, 2]]",[],"[robert, boulter, be, an, english, film, ,, te...","[Robert, Boulter, is, an, English, film, ,, te...","Robert Boulter is an English film , television...","{(Robert, Boulter): 1, (Robert, is): 2, (Rober...","{(0, 1): 1, (0, 2): 2, (0, 3): 4, (0, 4): 4, (..."
2,He had a guest @-@ starring role on the televi...,"{'toks': [['He', 'PRON'], ['had', 'VERB'], ['a...","[[He, PRON], [had, VERB], [a, DET], [guest, NO...","[O, O, O, O, O, O, O, O, O, B, Ī, O, O, O, O, O]","[[10, 11]]",[],"[he, have, a, guest, @-@, star, role, on, the,...","[He, had, a, guest, @-@, starring, role, on, t...",He had a guest @-@ starring role on the televi...,"{(He, had): 1, (He, a): 3, (He, guest): 3, (He...","{(0, 1): 1, (0, 2): 3, (0, 3): 3, (0, 4): 2, (..."
3,This was followed by a starring role in the pl...,"{'toks': [['This', 'PRON'], ['was', 'AUX'], ['...","[[This, PRON], [was, AUX], [followed, VERB], [...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-GROU...","[[14, 15], [24, 25]]",[],"[this, wa, follow, by, a, star, role, in, the,...","[This, was, followed, by, a, starring, role, i...",This was followed by a starring role in the pl...,"{(This, was): 2, (This, followed): 1, (This, b...","{(0, 1): 2, (0, 2): 1, (0, 3): 2, (0, 4): 4, (..."
4,He had a guest role in the television_series J...,"{'toks': [['He', 'PRON'], ['had', 'VERB'], ['a...","[[He, PRON], [had, VERB], [a, DET], [guest, AD...","[O, O, O, O, O, O, O, B-COMMUNICATION, Ī, O, B...","[[8, 9], [11, 12]]",[],"[he, have, a, guest, role, in, the, television...","[He, had, a, guest, role, in, the, television,...",He had a guest role in the television series J...,"{(He, had): 1, (He, a): 3, (He, guest): 3, (He...","{(0, 1): 1, (0, 2): 3, (0, 3): 3, (0, 4): 2, (..."


In [46]:
for i in df["syntactic_distance_idx"].iloc[0]:
    print(i)

(0, 1)
(0, 2)
(0, 3)
(1, 2)
(1, 3)
(2, 3)


In [34]:
df['token_map'] = df.apply(lambda x: tokenizations.get_alignments(x['tokens'], 
                                                                  strip_g(x['bpe_tokens']))[1], axis=1)

In [35]:
def list_to_index_dict(input_list):
    index_dict = {}
    
    for index, sublist in enumerate(input_list):
        for item in sublist:
            if item in index_dict:
                index_dict[item].append(index)
            else:
                index_dict[item] = [index]
    
    return index_dict

In [62]:
def map_syntactic_distance(df):
    new_map = {}
    mp = df["token_map_dict"]
    dist = df["syntactic_distance_idx"]
    for t0, t1 in dist:
        new_pair = []
        if t0 in mp:
            new_pair.append(mp[t0])
        if t1 in mp:
            new_pair.append(mp[t1])
        new_map[tuple(map(tuple, new_pair))] = dist[(t0, t1)]
        
    return new_map

In [58]:
df['token_map_dict'] = df['token_map'].apply(lambda x: list_to_index_dict(x))

In [ ]:
df["syntactic_distance_idx_mapped"] = df.apply(lambda x: map_syntactic_distance(x), axis=1)

In [52]:
df['syntactic_distance_idx'].iloc[0][(0,1)]

2

In [38]:
df.head()

,sentence,d,toks,tags,_,~,lemmas,tokens,sent,syntactic_distance_tok,syntactic_distance_idx,bpe_tokens,token_map,token_map_dict
0,= Robert_Boulter =,"{'toks': [['=', 'VERB'], ['Robert', 'PROPN'], ...","[[=, VERB], [Robert, PROPN], [Boulter, PROPN],...","[O, B-GROUP, Ī, O]","[[2, 3]]",[],"[=, robert, boulter, =]","[=, Robert, Boulter, =]",= Robert Boulter =,"{(=, Robert): 2, (=, Boulter): 1, (=, =): 1, (...","{(0, 1): 2, (0, 2): 1, (0, 3): 1, (1, 2): 1, (...","[=, ĠRobert, ĠB, oul, ter, Ġ=]","[[0], [1], [2], [2], [2], [3]]","{0: [0], 1: [1], 2: [2, 3, 4], 3: [5]}"
1,"Robert_Boulter is an English film , television...","{'toks': [['Robert', 'PROPN'], ['Boulter', 'PR...","[[Robert, PROPN], [Boulter, PROPN], [is, AUX],...","[B-PERSON, Ī, O, O, O, O, O, O, O, O, O, O]","[[1, 2]]",[],"[robert, boulter, be, an, english, film, ,, te...","[Robert, Boulter, is, an, English, film, ,, te...","Robert Boulter is an English film , television...","{(Robert, Boulter): 1, (Robert, is): 2, (Rober...","{(0, 1): 1, (0, 2): 2, (0, 3): 4, (0, 4): 4, (...","[Robert, ĠB, oul, ter, Ġis, Ġan, ĠEnglish, Ġfi...","[[0], [1], [1], [1], [2], [3], [4], [5], [6], ...","{0: [0], 1: [1, 2, 3], 2: [4], 3: [5], 4: [6],..."
2,He had a guest @-@ starring role on the televi...,"{'toks': [['He', 'PRON'], ['had', 'VERB'], ['a...","[[He, PRON], [had, VERB], [a, DET], [guest, NO...","[O, O, O, O, O, O, O, O, O, B, Ī, O, O, O, O, O]","[[10, 11]]",[],"[he, have, a, guest, @-@, star, role, on, the,...","[He, had, a, guest, @-@, starring, role, on, t...",He had a guest @-@ starring role on the televi...,"{(He, had): 1, (He, a): 3, (He, guest): 3, (He...","{(0, 1): 1, (0, 2): 3, (0, 3): 3, (0, 4): 2, (...","[He, Ġhad, Ġa, Ġguest, Ġ@, -, @, Ġstarring, Ġr...","[[0], [1], [2], [3], [4], [4], [4], [5], [6], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4, 5, 6],..."
3,This was followed by a starring role in the pl...,"{'toks': [['This', 'PRON'], ['was', 'AUX'], ['...","[[This, PRON], [was, AUX], [followed, VERB], [...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-GROU...","[[14, 15], [24, 25]]",[],"[this, wa, follow, by, a, star, role, in, the,...","[This, was, followed, by, a, starring, role, i...",This was followed by a starring role in the pl...,"{(This, was): 2, (This, followed): 1, (This, b...","{(0, 1): 2, (0, 2): 1, (0, 3): 2, (0, 4): 4, (...","[This, Ġwas, Ġfollowed, Ġby, Ġa, Ġstarring, Ġr...","[[0], [1], [2], [3], [4], [5], [6], [7], [8], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
4,He had a guest role in the television_series J...,"{'toks': [['He', 'PRON'], ['had', 'VERB'], ['a...","[[He, PRON], [had, VERB], [a, DET], [guest, AD...","[O, O, O, O, O, O, O, B-COMMUNICATION, Ī, O, B...","[[8, 9], [11, 12]]",[],"[he, have, a, guest, role, in, the, television...","[He, had, a, guest, role, in, the, television,...",He had a guest role in the television series J...,"{(He, had): 1, (He, a): 3, (He, guest): 3, (He...","{(0, 1): 1, (0, 2): 3, (0, 3): 3, (0, 4): 2, (...","[He, Ġhad, Ġa, Ġguest, Ġrole, Ġin, Ġthe, Ġtele...","[[0], [1], [2], [3], [4], [5], [6], [7], [8], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."


In [54]:
def map_weak_mwes_together(x):
    
    mapped_mwes = []
    for j, mwe in enumerate(x['_']):
        mapped_mwes.append([])
        for index in mwe:
            if index-1 in x['token_map_dict']:
                for val in x['token_map_dict'][index-1]:
                    mapped_mwes[j].append(val+1)
                
                
    return mapped_mwes
                   
    
def map_strong_mwes_together(x):
    
    mapped_mwes = []
    for j, mwe in enumerate(x['~']):
        mapped_mwes.append([])
        for index in mwe:
            
            if index-1 in x['token_map_dict']:
                for val in x['token_map_dict'][index-1]:
                    mapped_mwes[j].append(val+1)
                
                
    return mapped_mwes
                    

In [23]:
df['weak_mwe'] = df.apply(lambda x: map_weak_mwes_together(x),axis=1)
df['strong_mwe'] = df.apply(lambda x: map_strong_mwes_together(x),axis=1)